In [ ]:
!pip install ultralytics roboflow

from ultralytics import YOLO
import torch

print("Torch version:", torch.__version__)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/dataset"

Mounted at /content/drive


In [6]:
yaml_config = """
path: {project_path}
train: images/train
val: images/val

nc: 1
names: ["redwood"]
""".format(project_path=project_path)

with open("config.yaml", "w") as f:
    f.write(yaml_config)

In [ ]:
!pip install albumentations==1.3.1 --quiet
import cv2, os
import albumentations as A
import numpy as np

img_dir = project_path + "/images/train"
label_dir = project_path + "/labels/train"

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2,
                       rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.0, min_area=1e-6))

num_aug = 5

def clip_yolo_bbox(bbox):
    x, y, w, h = bbox
    x_min = max(0.0, x - w/2)
    y_min = max(0.0, y - h/2)
    x_max = min(1.0, x + w/2)
    y_max = min(1.0, y + h/2)
    x_new = (x_min + x_max) / 2
    y_new = (y_min + y_max) / 2
    w_new = x_max - x_min
    h_new = y_max - y_min
    return [x_new, y_new, w_new, h_new]

def filter_valid_bboxes(bboxes, classes, min_size=1e-6):
    valid_bboxes = []
    valid_classes = []
    for bbox, cls in zip(bboxes, classes):
        _, _, w, h = bbox
        if w > min_size and h > min_size:
            valid_bboxes.append(bbox)
            valid_classes.append(cls)
    return valid_bboxes, valid_classes

for img_file in os.listdir(img_dir):
    if not img_file.endswith(".jpg"):
        continue

    img_path = os.path.join(img_dir, img_file)
    label_path = os.path.join(label_dir, img_file.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        continue

    image = cv2.imread(img_path)

    bboxes, class_labels = [], []
    with open(label_path) as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.split())
            bboxes.append([x, y, bw, bh])
            class_labels.append(int(cls))

    bboxes = [clip_yolo_bbox(b) for b in bboxes]

    for i in range(num_aug):
        try:
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        except ValueError:
            continue

        aug_img = augmented['image']
        aug_bboxes, aug_classes = filter_valid_bboxes(augmented['bboxes'], augmented['class_labels'])

        if len(aug_bboxes) == 0:
            continue

        aug_img_name = img_file.replace(".jpg", f"_aug{i}.jpg")
        cv2.imwrite(os.path.join(img_dir, aug_img_name), aug_img)

        aug_label_name = aug_img_name.replace(".jpg", ".txt")
        with open(os.path.join(label_dir, aug_label_name), "w") as f:
            for cls, (x, y, bw, bh) in zip(aug_classes, aug_bboxes):
                f.write(f"{cls} {x} {y} {bw} {bh}\n")

In [ ]:
model = YOLO("yolov8n.pt")
model.train(
    data="config.yaml",
    epochs=20,
    imgsz=416,
    batch=16,
    project="redwood_detection",
    workers=2,
    cache=True,
    name="yolov8n_redwood"
)

In [ ]:
model = YOLO("/content/redwood_detection/yolov8n_redwood8/weights/best.pt")
results = model.predict(source="/content/drive/MyDrive/dataset/images/train", imgsz=640, conf=0.25)


In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
results = model.predict(source=f"{project_path}/images/val", save=True, conf=0.25)

In [ ]:
model.export(format="onnx")